In [ ]:
from astroquery.mast import Observations
from astropy.io import fits
import numpy as np

In [ ]:
tic, sector, planetNum = 70763084, 2, 1
localPath = "/Users/vkostov/.eleanor"

In [ ]:
class TessAstroqueryArchive(object):
    
    def __init__(self, cachePath="./"):
        
        self.cachePath=cachePath

    def parse(self, localUrl, *args, **kwargs):
        """Load data using favourite IO class
        
        Assumes that localUrl exists on disk. Default file format
        for MAST is FITS, but sub-classes can override this method
        for other file types as necessary
        
        All optional arguments are passed into astropy.io.fits.getdata
        
        """
        if 'ext' in kwargs and kwargs['ext'] == 0:
            try:
                retVal = fits.getheader(localUrl, *args, **kwargs)
            except IOError as  e:
                raise e
        else:
            try:
                retVal = fits.getdata(localUrl, *args, **kwargs)
            except IOError as  e:
                raise e

        return retVal
    
    
    def getPostageStampObservationId(self, sector, ticid):
        """
        Using Astroquery find the two minute obsid.
        ticid = integer tic id
        sector = integer sector number
        """
        
        obs_query_string = "tess*-s%04i-%016i*" % (sector, ticid)
        obsTable = Observations.query_criteria(mission="TESS", obs_id=obs_query_string)
        
#        print(obsTable)#['obs_id'])
#        xxxx

        #@todo check only one element returned
        #self.obsid = obsTable['obsid']
        
        return obsTable['obsid']
   
    
    def getOneFileFromMastObs(self, ticid, sector, fileType, *args, **kwargs):
        """
        Generic function to retrive a particular file type from a
        CAOM observation at the MAST.
        Private Function --  Only used by this class.
        """
        
        if fileType not in ['LC','TP','DVT']:
            raise ValueError("Requested file Type (%s) not in allowed list (LC,TP,DVT)."\
                             % fileType)
        
        obsid = self.getPostageStampObservationId(sector, ticid)

        manifest=Observations.download_products(obsid,\
                                            productSubGroupDescription=[fileType],\
                                            mrp_only=False, extension="fits",
                                            download_dir=self.cachePath)

        localUrl=manifest['Local Path'][0]# if fileType == 'TP' else manifest['Local Path'][1]

#        print(fileType, obsid)

        return self.parse(localUrl, *args, **kwargs)
    
    
    def getLightcurve(self, ticid, sector, *args, **kwargs):
        """
        Get the light curve file for a given TIC Id and Sector.
        
        All optional arguments are passed into `astropy.io.fits.getdata`
        including header = True        
        
        """

        return self.getOneFileFromMastObs(ticid,sector,'LC', *args, **kwargs)
        
        
       
    
    def getDvt(self, ticid, sector, *args, **kwargs):
        """
        Get the Data Validation time series file for a given TIC Id and Sector.
        
        All optional arguments are passed into `astropy.io.fits.getdata`
        including header = True  
        ext = planetNumber
        
        """

        return self.getOneFileFromMastObs(ticid,sector,'DVT', *args, **kwargs)
        
    
    def getTPF(self, ticid, sector, *args, **kwargs):
        """
        Get the Target Pixel File for a given TIC Id and Sector.
        
        All optional arguments are passed into `astropy.io.fits.getdata`
        including header = True
        """
        
        return self.getOneFileFromMastObs(ticid,sector,'TP', *args, **kwargs)

In [ ]:
def getTargetPixelArrayFromFits(fits, hdr, column='FLUX'):
    nRows = int(hdr['NAXIS2'])
    #All imgs are the same size, so it doesn't matter which TDIM
    #I pick here.
    shape = eval(hdr['TDIM5'])
    #The reverse puts the columns on the x-axis and row
    #on the y-axis when you call mp.imshow()
    shape = tuple(reversed(shape))

    tpfArray = np.empty((nRows, shape[0], shape[1]))

    for i, cadence in enumerate(fits[column]):
        tpfArray[i,:,:] = cadence.reshape(shape)

    return tpfArray

In [ ]:
ar = TessAstroqueryArchive(localPath)

dvt, hdr = ar.getLightcurve(tic, sector, ext=planetNum, header=True)
flux = dvt['SAP_FLUX']#dvt['PDCSAP_FLUX']

tpf_, hdr_tpf = ar.getTPF(tic, sector, ext=planetNum, header=True)
cube = getTargetPixelArrayFromFits(tpf_, hdr_tpf)

time = dvt['TIME']
flags = np.isnan(flux)